### Alphabet Soup Charity Optimization Model

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' 
application_df = application_df.drop([ "EIN", "NAME", "SPECIAL_CONSIDERATIONS","AFFILIATION", "USE_CASE","ORGANIZATION"], axis=1)
application_df.head()


,APPLICATION_TYPE,CLASSIFICATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,1,0,5000,1
1,T3,C2000,1,1-9999,108590,1
2,T5,C3000,1,0,5000,0
3,T3,C2000,1,10000-24999,6692,1
4,T3,C1000,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
unique_counts = application_df.nunique()
unique_counts


APPLICATION_TYPE      17
CLASSIFICATION        71
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_counts[app_counts < 600].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(
        app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = class_counts.loc[class_counts > 1]
class_counts_gt1.head()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(class_counts[class_counts > 2000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(
        cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


Other    28237
C3000     1918
C2100     1883
C7000      777
C1700      287
         ...  
C4120        1
C2561        1
C1245        1
C8210        1
C2150        1
Name: CLASSIFICATION, Length: 69, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dum = application_df.dtypes[application_df.dtypes=='object'].index

dum_data = pd.get_dummies(application_df, columns=application_dum)

dum_data. head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
X = dum_data.drop(['IS_SUCCESSFUL'], axis=1)
y = dum_data['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 30


nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                7120      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 30)                930       
                                                                 
 dense_3 (Dense)             (None, 1)                 31        
                                                                 
Total params: 10,511
Trainable params: 10,511
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
# Train the model
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Encode the target labels
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

fit_model = nn_model.fit(X_train_scaled, y_train_encoded, epochs=30)



Epoch 1/30
804/804 [==============================] - 7s 6ms/step - loss: 0.6422 - accuracy: 0.6043
Epoch 2/30
804/804 [==============================] - 11s 14ms/step - loss: 0.6357 - accuracy: 0.6102
Epoch 3/30
804/804 [==============================] - 7s 9ms/step - loss: 0.6349 - accuracy: 0.6145
Epoch 4/30
804/804 [==============================] - 4s 4ms/step - loss: 0.6341 - accuracy: 0.6151
Epoch 5/30
804/804 [==============================] - 3s 4ms/step - loss: 0.6337 - accuracy: 0.6132
Epoch 6/30
804/804 [==============================] - 2s 3ms/step - loss: 0.6336 - accuracy: 0.6143
Epoch 7/30
804/804 [==============================] - 1s 2ms/step - loss: 0.6333 - accuracy: 0.6150
Epoch 8/30
804/804 [==============================] - 1s 2ms/step - loss: 0.6330 - accuracy: 0.6152
Epoch 9/30
804/804 [==============================] - 2s 3ms/step - loss: 0.6329 - accuracy: 0.6153
Epoch 10/30
804/804 [==============================] - 2s 2ms/step - loss: 0.6322 - accuracy: 0.61

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6374 - accuracy: 0.6103 - 514ms/epoch - 2ms/step
Loss: 0.6374341249465942, Accuracy: 0.610262393951416


In [16]:
# Export our model to HDF5 file
fit_model.model.save("AlphabetSoupCharity_Optimization2.h5")

from google.colab import files

files.download("AlphabetSoupCharity_Optimization2.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>